# **Gradio 사용해 보기**

---


- 💡 **NOTE**
    - 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.


---

## **Gradio란?**

- 머신러닝 모델이나 데이터 처리 함수를 웹 인터페이스로 아주 쉽게 시각화·공유할 수 있도록 도와주는 오픈소스 라이브러리
- 즉, **AI 모델을 웹 데모로 즉시 실행·테스트할 수 있게 해주는 도구**
- 현재는 Hugging Face에서 관리
    - AI 연구자들은 모델을 만들고 논문을 쓰는 데는 익숙했지만, 이 모델이 실제로 어떻게 작동하는지 보여주는 '데모'를 만드는 데는 어려움을 겪었음
    - 2020년, Abubakar Abid를 비롯한 연구팀이 이러한 문제를 해결하기 위해 Gradio를 개발
    - 2021년, Hugging Face는 Gradio를 인수
    - HF Spaces(모델 데모를 호스팅하는 플랫폼)의 표준 UI 빌더
- **주요 기능**
    - **(극도의 단순함)** : **gr.Interface(...)** 몇 줄이면 UI가 완성
    - **(다양한 컴포넌트)** : 텍스트 박스, 이미지 업로드, 슬라이더, 드롭다운, 오디오 녹음, 비디오 등 AI 모델에 필요한 거의 모든 입출력 컴포넌트를 미리 제공
    - **(자동 공유 링크)** :  **.launch(share=True) 옵션** 하나만 추가하면, 방화벽이나 복잡한 설정 없이도 72시간 동안 유효한 공개 URL을 생성
    - **(유연성 및 확장성)** : **gr.Blocks**라는 기능을 통해 **챗봇 UI, 탭(Tab) 구분, 복잡한 레이아웃 등** 고도로 커스터마이징된 **웹 앱도 구축할 수 있음**
    - **(임베딩)** : Jupyter Notebook, Google Colab, 웹사이트 등 어디에나 쉽게 데모 창을 삽입할 수 있음
- 사용 언어 : Python (단, 브라우저 UI는 HTML/JS로 자동 생성됨)

### **작동 원리**
Gradio는 다음 세 단계를 거쳐 작동합니다:

1. **함수 정의**:
    - 예를 들어, 텍스트를 입력하면 요약 결과를 반환하는 predict(text) 함수를 만든다.
2. **입출력 형식 지정**:
    - gr.Interface(fn=predict, inputs="text", outputs="text")
3. **실행 및 공유**:
    - interface**.launch()** 호출 시
    - → 자동으로 로컬 서버(예: http://127.0.0.1:7860)가 열리고
    - → 웹 브라우저에서 GUI 형태로 모델을 테스트할 수 있음

## **라이브러리 설치**

In [1]:
!pip install gradio

### **예제: 인삿말 표현(Text → Text)**

In [2]:
import gradio as gr

# 1. Python 함수 정의 (우리가 만든 모델 또는 기능)
def greet(name):
    """입력받은 이름에 인사를 덧붙여 반환합니다."""
    if not name:
        return "이름을 입력해주세요!"
    return f"안녕하세요, {name}님! 만나서 반갑습니다. 👋"

# 2. Gradio 인터페이스 생성
# fn: 연결할 함수
# inputs: 입력 컴포넌트 (gr.Textbox를 사용하고, 라벨과 플레이스홀더 지정)
# outputs: 출력 컴포넌트 (gr.Textbox)
iface = gr.Interface(
    fn=greet,
    inputs=gr.Textbox(lines=1, label="이름", placeholder="여기에 이름을 입력하세요..."),
    outputs=gr.Textbox(label="인사말"),
    title="Gradio 인사 로봇",
    description="이름을 입력하면 로봇이 인사를 해줍니다."
)

# 3. 인터페이스 실행 (share=True로 설정하면 외부 공유 링크가 생성됩니다)
iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d773e317c8b0954bf1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### **예제: 대문자 변환**

In [3]:
import gradio as gr

# 간단한 텍스트 변환 함수
def to_upper(text):
    return text.upper()

# Gradio 인터페이스 생성
demo = gr.Interface(
    fn=to_upper,
    inputs="text",   # 입력: 텍스트박스
    outputs="text",  # 출력: 텍스트박스
    title="대문자 변환기",
    description="입력한 문장을 모두 대문자로 바꿉니다."
)

# 실행
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b7c350496c8cef0e50.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### **예제 : 이미지 필터 적용하기** (Image → Image)

In [4]:
!pip install pillow

In [5]:
import gradio as gr
from PIL import Image, ImageFilter

# 1. Python 함수 정의 (이미지를 받아서 흑백으로 변경)
def to_black_and_white(input_image):
    """PIL Image 객체를 입력받아 흑백(Grayscale)으로 변환하여 반환합니다."""
    if input_image is None:
        return None

    # PIL Image 객체로 변환 (Gradio는 numpy array로 이미지를 전달)
    # img = Image.fromarray(input_image) # Gradio 버전에 따라 이 줄이 필요할 수 있음

    # 흑백으로 변환
    bw_image = input_image.convert("L")
    return bw_image

# 2. Gradio 인터페이스 생성
# inputs: "image" (이미지 업로드 컴포넌트가 자동으로 생김)
# outputs: "image" (이미지 출력 컴포넌트가 생김)
iface = gr.Interface(
    fn=to_black_and_white,
    inputs=gr.Image(type="pil", label="원본 이미지 업로드"), # type="pil"로 받음
    outputs=gr.Image(type="pil", label="흑백 변환 이미지"),
    title="이미지 흑백 변환기",
    description="이미지를 업로드하면 흑백으로 바꿔줍니다."
)

# 3. 인터페이스 실행
iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8b2c12d2b5089f8b18.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)




---



## **Gradio 사용 예제**

### **예제 : (Blocks 활용 챗봇) Hugging face 모델 사용하여 챗봇 만들기**

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import gradio as gr

# ============================================
# 1. 모델 및 토크나이저 로드
# ============================================
model_id = "Qwen/Qwen2.5-3B-Instruct"

print("모델을 로딩중입니다... 잠시만 기다려주세요.")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
    attn_implementation="eager"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=500,
    do_sample=True,  # 다양한 응답을 위해 True로 변경
    temperature=0.7,  # 창의성 조절 (0.1~1.0)
    top_p=0.9
)

output = pipe('[모델 테스트] 너의 이름은 뭐야?')
print(output[0]["generated_text"])

print("모델 로딩 완료!")


# ============================================
# 2. 챗봇 응답 생성 함수
# ============================================
def chat_with_model(user_message, history):
    """
    사용자 메시지를 받아서 모델의 응답을 생성하는 함수

    Args:
        user_message (str): 사용자가 입력한 메시지
        history (list): 이전 대화 기록 [[user_msg, bot_msg], ...]

    Returns:
        str: 모델이 생성한 응답
    """
    # 대화 히스토리를 모델 형식으로 변환
    messages = []

    # 이전 대화를 추가 (최근 5턴만 유지하여 토큰 제한 방지)
    for user_msg, bot_msg in history[-5:]:
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": bot_msg})

    # 현재 사용자 메시지 추가
    messages.append({"role": "user", "content": user_message})

    # 모델로 응답 생성
    try:
        output = pipe(messages)
        bot_response = output[0]["generated_text"]
        return bot_response
    except Exception as e:
        return f"오류가 발생했습니다: {str(e)}"


# ============================================
# 3. Gradio 인터페이스 구성
# ============================================
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 🤖 Qwen2.5-3B 챗봇
        ### AI와 자유롭게 대화해보세요!
        """
    )

    # 채팅 인터페이스
    chatbot = gr.Chatbot(
        height=500,
        label="대화창",
        bubble_full_width=False
    )

    # 사용자 입력창
    with gr.Row():
        msg = gr.Textbox(
            label="메시지 입력",
            placeholder="질문을 입력하세요...",
            scale=4
        )
        submit_btn = gr.Button("전송", scale=1, variant="primary")

    # 초기화 버튼
    clear_btn = gr.Button("대화 초기화")

    # 예시 질문
    gr.Examples(
        examples=[
            "안녕하세요! 자기소개 해주세요.",
            "파이썬으로 피보나치 수열을 구하는 코드를 알려주세요.",
            "인공지능의 미래에 대해 어떻게 생각하시나요?"
        ],
        inputs=msg
    )

    # ============================================
    # 4. 이벤트 핸들러
    # ============================================
    def user_submit(user_message, history):
        """사용자 메시지를 받아 히스토리에 추가"""
        return "", history + [[user_message, None]]

    def bot_response(history):
        """봇의 응답을 생성하여 히스토리에 추가"""
        user_message = history[-1][0]
        bot_message = chat_with_model(user_message, history[:-1])
        history[-1][1] = bot_message
        return history

    # 전송 버튼 클릭 이벤트
    submit_btn.click(
        user_submit,
        inputs=[msg, chatbot],
        outputs=[msg, chatbot]
    ).then(
        bot_response,
        inputs=[chatbot],
        outputs=[chatbot]
    )

    # 엔터키 입력 이벤트
    msg.submit(
        user_submit,
        inputs=[msg, chatbot],
        outputs=[msg, chatbot]
    ).then(
        bot_response,
        inputs=[chatbot],
        outputs=[chatbot]
    )

    # 초기화 버튼 클릭 이벤트
    clear_btn.click(lambda: None, None, chatbot, queue=False)


# ============================================
# 5. 앱 실행
# ============================================
if __name__ == "__main__":
    demo.launch()
    # demo.launch(
    #     share=False,  # True로 설정하면 공개 URL 생성
    #     server_name="0.0.0.0"  # 외부 접속 허용
    # )

모델을 로딩중입니다... 잠시만 기다려주세요.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda


 '알파고'와 '디즈니'가 만든 AI 아트작품, 정말 예쁘다 - 코리아투데이
[모델 테스트] 너의 이름은 뭐야? '알파고'와 '디즈니'가 만든 AI 아트작품, 정말 예쁘다
승인 2018.09.25 17:47
"이미지 생성하는 AI, 세상을 바꾼다"
[서울=코리아투데이] 김성준 기자 = AI(AI 인공지능)를 활용해 이미지를 생성하는 기술이 발달하면서, 그에 대한 관심이 높아지고 있다.
AI는 이미지 생성을 위해 사용되는 알고리즘을 통해 특정한 이미지나 스타일을 재현하거나 새로운 이미지를 생성하는 기술이다. 이 기술이 발전함에 따라 인공지능이 인간의 감각과 창의력을 연동하며 인간의 지평을 넓혀가고 있다. AI 기술이 발전하면서 이미지 생성 기술도 빠르게 발전하고 있다.
AI의 이미지 생성 기술 중 하나로 유명한 것이 'GAN(Generative Adversarial Network)'이다. GAN은 두 개의 네트워크를 이용한 신경망으로, 하나는 '생성자'로, 다른 하나는 '판별자'로 불린다. 생성자는 실제 이미지나 데이터를 모방하고 판별자는 생성된 이미지를 분류한다. 두 네트워크는 서로 대결적으로 작동하며, 생성자가 더 좋은 이미지를 만들어내면 판별자가 그 이미지를 더 잘 판단할 수 있게 된다. 이렇게 반복해서 진행하면 생성자가 실제 이미지를 만들 수 있는 능력을 갖추게 된다.
GAN을 이용해 여러 이미지 생성 프로그램이 만들어졌는데, 이중 가장 유명한 것은 '알파고'와 '디즈니'가 함께 만든 'Deep Dream Generator'이다. 이 프로그램은 컴퓨터가 사람의 시각적 정보를 인식하는 방식을 바탕으로 이미지를 생성해낸다. 'Deep Dream Generator'는 2015년 5월 출시되었고, 이 프
모델 로딩 완료!


/tmp/ipython-input-2660095968.py:84: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-2660095968.py:84: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ed96f08e16eeaa6e1b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
demo.launch()

### **예제 : (탭 사용) Task별 모델 사용하기**

In [ ]:
!pip list | grep -E  "gradio|transformers|torch"


In [7]:
# 1. ★★★ 수정된 라이브러리 설치 ★★★
# torch를 설치 목록에서 제외하여 Colab의 기본 버전을 사용합니다.
!pip install -qU gradio transformers "transformers[sentencepiece]"

print("\n--- 라이브러리 설치 완료 ---")
print("Gradio와 Transformers가 설치/업그레이드되었습니다.")
print("Colab의 기본 torch 버전을 사용합니다.")

# (중요)
# 라이브러리 설치 후, '런타임 > 런타임 다시 시작'을 수동으로 실행하거나
# 아래 코드를 통해 자동으로 런타임을 재시작할 수 있습니다.
# 런타임 재시작 후, 이 셀을 다시 실행하지 말고 [아래 2번 셀]부터 실행하세요.
# import os
# os.kill(os.getpid(), 9)


--- 라이브러리 설치 완료 ---
Gradio와 Transformers가 설치/업그레이드되었습니다.
Colab의 기본 torch 버전을 사용합니다.


In [ ]:
!pip list | grep -E  "gradio|transformers|torch"


In [8]:
# 2. (런타임 재시작 후) 여기부터 실행하세요.
import gradio as gr
from transformers import pipeline, logging
import torch
import warnings

# 3. 불필요한 경고 메시지 숨기기
logging.set_verbosity_error()
warnings.filterwarnings("ignore")

# 4. 모델 리스트 정의
cls_models = [
    "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli",
    "joeddav/xlm-roberta-large-xnli",
    "MoritzLaurer/multilingual-MiniLMv2-L6-mnli-xnli"
]
gen_models = [
    "EleutherAI/polyglot-ko-1.3b",
    "google/mt5-small",
    "Qwen/Qwen2.5-3B-Instruct"
]
sum_models = [
    "Qwen/Qwen2.5-0.5B-Instruct",
    "gogamza/kobart-summarization",
    "EleutherAI/polyglot-ko-12.8b"
    # "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"
]

# 유틸 함수 추가
from transformers import AutoConfig

def is_encoder_decoder(model_name: str) -> bool:
    try:
        cfg = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
        return bool(getattr(cfg, "is_encoder_decoder", False))
    except Exception:
        # 구성 정보를 못 읽으면 일단 디코더-only로 간주
        return False


# 5. 파이프라인 캐시
PIPELINES_CACHE = {}

def get_pipeline(task, model_name):
    # ⚠️ 요약 탭에서 디코더-only 모델을 고르면 text-generation으로 우회
    effective_task = task
    if task == "summarization" and not is_encoder_decoder(model_name):
        effective_task = "text-generation"

    key = (effective_task, model_name)
    if key not in PIPELINES_CACHE:
        print(f"\n[{effective_task}] '{model_name}' 모델을 새로 로드합니다...")
        try:
            device = 0 if torch.cuda.is_available() else -1
            PIPELINES_CACHE[key] = pipeline(
                effective_task,
                model=model_name,
                repetition_penalty=1.2, # 🔑 핵심: 반복 패널티 (1.0~2.0)
                no_repeat_ngram_size=3, # 🔑 3-gram 이상 반복 금지,
                trust_remote_code=True,
                device=device
            )
            print(f"'{model_name}' 로드 완료.")
        except Exception as e:
            print(f"모델 로드 중 오류 발생: {e}")
            gr.Warning(f"모델 로드 실패: {model_name}. {e}")
            return None

    return PIPELINES_CACHE[key]


# 6. 각 탭 기능 함수

# --- (버그 수정된) 텍스트 분류 함수 ---
def classify_text(model_name, text_input, labels_input):
    if not text_input or not labels_input:
        gr.Warning("텍스트와 라벨을 모두 입력하세요.")
        return {}

    pipe = get_pipeline(task="zero-shot-classification",model_name=model_name)
    if pipe is None:
        return {}

    candidate_labels = [label.strip() for label in labels_input.split(',')]

    try:
        result = pipe(text_input, candidate_labels=candidate_labels)

        # 'labels' 리스트와 'scores' 리스트를 zip으로 묶어 딕셔너리 생성
        formatted_result = {label: round(score, 3)
                            for label, score in zip(result['labels'], result['scores'])}

        return formatted_result
    except Exception as e:
        gr.Error(f"분류 중 오류가 발생했습니다: {e}")
        return {}

def generate_text(model_name, prompt, max_len):
    if not prompt:
        gr.Warning("프롬프트를 입력하세요.")
        return ""

    pipe = get_pipeline(task="text-generation", model_name=model_name)
    if pipe is None:
        return ""

    try:
        if pipe.model.config.pad_token_id is None:
            pipe.model.config.pad_token_id = pipe.tokenizer.eos_token_id

        result = pipe(prompt, max_new_tokens=int(max_len), num_return_sequences=1)
        return result[0]['generated_text']
    except Exception as e:
        gr.Error(f"생성 중 오류: {e}")
        return ""

def summarize_text(model_name, text_input, min_len, max_len):
    if not text_input:
        gr.Warning("요약할 원본 텍스트를 입력하세요.")
        return ""

    pipe = get_pipeline(task="summarization", model_name=model_name)
    if pipe is None:
        return ""

    try:
        # 디코더-only 우회: text-generation 파이프라인이면 프롬프트 요약
        if getattr(pipe, "task", "") == "text-generation":
            if pipe.model.config.pad_token_id is None:
                pipe.model.config.pad_token_id = pipe.tokenizer.eos_token_id

            prompt = (
                "다음 글의 핵심을 3~5문장으로 간결하게 한국어 요약하세요.\n\n"
                f"[원문]\n{text_input}\n\n[요약]"
            )
            out = pipe(prompt, max_new_tokens=int(max_len), num_return_sequences=1)
            return out[0]["generated_text"].split("[요약]")[-1].strip()

        # 정상(Seq2Seq) 경로
        result = pipe(
            text_input,
            min_length=int(min_len),
            max_length=int(max_len)
        )
        return result[0]["summary_text"]
    except Exception as e:
        gr.Error(f"요약 중 오류: {e}")

        return ""


# 7. Gradio UI 구성 (초기화 버튼 추가됨)
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🤖 다중 작업(Multi-Task) AI 데모 (Gradio + 🤗)")
    gr.Markdown("각 탭을 눌러 원하는 AI 작업을 테스트해보세요.")

    # --- 탭 1: Text Classification ---
    with gr.Tab("text-classification"):
        gr.Markdown("## 📰 텍스트 분류 (Zero-Shot)")
        gr.Markdown("문장이 어떤 카테고리에 속하는지 분류합니다.")

        model_cls = gr.Dropdown(label="분류 모델 선택", choices=cls_models, value=cls_models[0])
        with gr.Row():
            text_cls = gr.Textbox(label="분류할 텍스트", placeholder="예: 오늘 코스피 지수가 5% 급등했습니다.", scale=3)
            labels_cls = gr.Textbox(label="후보 라벨 (쉼표로 구분)", value="정치, 경제, 사회, IT, 스포츠", scale=2)

        output_cls = gr.Label(label="분류 결과 (확률)")

        # --- ★ 초기화 버튼 추가 ★ ---
        with gr.Row():
            btn_cls = gr.Button("분류 실행", variant="primary", scale=4)
            # gr.ClearButton을 추가하고, 초기화할 컴포넌트 리스트를 지정합니다.
            clear_btn_cls = gr.ClearButton(
                value="🔄 입력/결과 초기화",
                components=[text_cls, labels_cls, output_cls],
                scale=1
            )

        btn_cls.click(fn=classify_text, inputs=[model_cls, text_cls, labels_cls], outputs=[output_cls])

    # --- 탭 2: Text Generation ---
    with gr.Tab("text-generation"):
        gr.Markdown("## ✍️ 텍스트 생성")
        gr.Markdown("프롬프트(시작 단어)를 기반으로 다음 문장을 생성합니다.")

        model_gen = gr.Dropdown(label="생성 모델 선택", choices=gen_models, value=gen_models[0])
        text_gen = gr.Textbox(label="프롬프트 (시작 단어)", placeholder="예: 대한민국의 수도는")
        slider_gen = gr.Slider(label="최대 생성 토큰 수", minimum=10, maximum=200, value=50, step=10)

        output_gen = gr.Textbox(label="생성된 텍스트", lines=5, interactive=False)

        # --- ★ 초기화 버튼 추가 ★ ---
        with gr.Row():
            btn_gen = gr.Button("생성 시작", variant="primary", scale=4)
            clear_btn_gen = gr.ClearButton(
                value="🔄 입력/결과 초기화",
                # 탭 2의 컴포넌트들을 지정합니다.
                components=[text_gen, slider_gen, output_gen],
                scale=1
            )

        btn_gen.click(fn=generate_text, inputs=[model_gen, text_gen, slider_gen], outputs=[output_gen])

    # --- 탭 3: Text Summarization ---
    with gr.Tab("text-summarization"):
        gr.Markdown("## 📑 텍스트 요약")
        gr.Markdown("긴 문서를 짧게 요약합니다.")

        model_sum = gr.Dropdown(label="요약 모델 선택", choices=sum_models, value=sum_models[0])
        text_sum = gr.Textbox(label="요약할 원본 텍스트", lines=10, placeholder="여기에 긴 뉴스 기사나 문서를 붙여넣으세요...")
        with gr.Row():
            slider_min_sum = gr.Slider(label="최소 요약 길이", minimum=10, maximum=100, value=20, step=5)
            slider_max_sum = gr.Slider(label="최대 요약 길이", minimum=50, maximum=300, value=100, step=10)

        output_sum = gr.Textbox(label="요약된 텍스트", lines=5, interactive=False)

        # --- ★ 초기화 버튼 추가 ★ ---
        with gr.Row():
            btn_sum = gr.Button("요약 실행", variant="primary", scale=4)
            clear_btn_sum = gr.ClearButton(
                value="🔄 입력/결과 초기화",
                # 탭 3의 컴포넌트들을 지정합니다.
                components=[text_sum, slider_min_sum, slider_max_sum, output_sum],
                scale=1
            )

        btn_sum.click(fn=summarize_text, inputs=[model_sum, text_sum, slider_min_sum, slider_max_sum], outputs=[output_sum])

# 8. Gradio 앱 실행
print("Gradio 앱을 실행합니다. (첫 모델 로드에 다소 시간이 걸릴 수 있습니다...)")
demo.launch(debug=True)

Gradio 앱을 실행합니다. (첫 모델 로드에 다소 시간이 걸릴 수 있습니다...)
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ce0ce39186095c63e0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



[zero-shot-classification] 'MoritzLaurer/mDeBERTa-v3-base-mnli-xnli' 모델을 새로 로드합니다...


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

'Qwen/Qwen2.5-0.5B-Instruct' 로드 완료.
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d773e317c8b0954bf1.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://b7c350496c8cef0e50.gradio.live
Killing tunnel 127.0.0.1:7862 <> https://8b2c12d2b5089f8b18.gradio.live
Killing tunnel 127.0.0.1:7863 <> https://ed96f08e16eeaa6e1b.gradio.live
Killing tunnel 127.0.0.1:7864 <> https://ce0ce39186095c63e0.gradio.live




---



###  **예제 : 코드 최적화하기**

1. **모델 양자화 (Quantization)**
    - 역사적 배경: 2015년 구글이 TensorFlow Lite에서 처음 도입. 딥러닝 모델의 가중치를 FP32(32비트 부동소수점)에서 INT8(8비트 정수)로 변환하여 모델 크기를 줄이고 추론 속도를 높이는 기법입니다.
2. **사전 로딩 (Preloading)**
    - 모델 로딩은 I/O 집약적 작업으로, 앱 시작 시 미리 로드하면 사용자의 첫 요청 대기 시간을 크게 줄일 수 있습니다.
3. **JIT 컴파일 (torch.compile)**
    - 역사적 배경: PyTorch 2.0 (2023년)에서 도입. TorchDynamo를 사용하여 Python 코드를 최적화된 C++ 코드로 변환합니다.


|최적화 기법|속도 개선 효과|메모리 절감|구현 난이도|적용 시점|
|---|---|---|---|---|
|**모델 양자화 (INT8)**|2~4배 빠름|50~75% 감소|중간|모델 로드 시|
|**사전 로딩 (Preloading)**|첫 실행 3~10배 빠름|변화 없음|쉬움|앱 시작 시|
|**torch.compile()**|1.5~2배 빠름|약간 증가|쉬움|모델 로드 후|
|**배치 처리**|2~5배 빠름 (다중 입력)|변화 없음|중간|추론 시|
|**GPU 메모리 최적화**|1.2~1.5배 빠름|30~50% 감소|어려움|전체 과정|

In [ ]:
# ============================================
# 🚀 최적화된 Gradio Multi-Task AI 데모
# ============================================

import gradio as gr
from transformers import pipeline, logging, AutoConfig
import torch
import warnings
from functools import lru_cache
import gc

# 1. 경고 메시지 숨기기
logging.set_verbosity_error()
warnings.filterwarnings("ignore")

# 2. bitsandbytes 조건부 import (없어도 작동)
try:
    from transformers import BitsAndBytesConfig
    QUANTIZATION_AVAILABLE = True
except ImportError:
    QUANTIZATION_AVAILABLE = False
    print("⚠️ bitsandbytes 미설치 (양자화 기능 비활성화)")

# 3. GPU 메모리 최적화 설정
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.backends.cudnn.benchmark = True
    print(f"✅ GPU 사용 가능: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ CPU 모드로 실행됩니다.")

# 4. 양자화 설정 (사용 가능한 경우만)
quantization_config = None
if QUANTIZATION_AVAILABLE:
    try:
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4"
        )
    except:
        QUANTIZATION_AVAILABLE = False

# 5. 최적화된 모델 리스트
cls_models = [
    "MoritzLaurer/multilingual-MiniLMv2-L6-mnli-xnli",
    "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli",
    "joeddav/xlm-roberta-large-xnli"
]
gen_models = [
    "google/mt5-small",
    "EleutherAI/polyglot-ko-1.3b",
    "Qwen/Qwen2.5-3B-Instruct"
]
sum_models = [
    "Qwen/Qwen2.5-0.5B-Instruct",
    "gogamza/kobart-summarization",
    "EleutherAI/polyglot-ko-12.8b"
]

# 6. 파이프라인 캐시
PIPELINES_CACHE = {}
PRELOADED_MODELS = set()

@lru_cache(maxsize=32)
def is_encoder_decoder(model_name: str) -> bool:
    try:
        cfg = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
        return bool(getattr(cfg, "is_encoder_decoder", False))
    except Exception:
        return False

def get_pipeline_optimized(task, model_name, use_quantization=False):
    effective_task = task
    if task == "summarization" and not is_encoder_decoder(model_name):
        effective_task = "text-generation"

    key = (effective_task, model_name)

    if key not in PIPELINES_CACHE:
        print(f"\n🔄 [{effective_task}] '{model_name}' 모델 로딩 중...")

        try:
            device = 0 if torch.cuda.is_available() else -1

            pipe_kwargs = {
                "task": effective_task,
                "model": model_name,
                "trust_remote_code": True,
                "device": device,
            }

            # 양자화 적용 (가능한 경우만)
            if use_quantization and QUANTIZATION_AVAILABLE and torch.cuda.is_available():
                pipe_kwargs["model_kwargs"] = {
                    "quantization_config": quantization_config,
                    "torch_dtype": torch.float16,
                }

            pipe = pipeline(**pipe_kwargs)

            # torch.compile 적용 (가능한 경우만)
            if hasattr(torch, 'compile') and torch.cuda.is_available():
                try:
                    pipe.model = torch.compile(pipe.model, mode="reduce-overhead")
                    print("   ✅ torch.compile 적용 완료")
                except:
                    pass

            PIPELINES_CACHE[key] = pipe
            print(f"   ✅ '{model_name}' 로드 완료!")

        except Exception as e:
            print(f"   ❌ 모델 로드 실패: {e}")
            gr.Warning(f"모델 로드 실패: {model_name}")
            return None

    return PIPELINES_CACHE[key]

# 7. 앱 시작 시 기본 모델 사전 로딩
def preload_models():
    print("\n🚀 기본 모델 사전 로딩 시작...")
    default_models = [
        ("zero-shot-classification", cls_models[0]),
        ("text-generation", gen_models[0]),
        ("summarization", sum_models[0])
    ]

    for task, model in default_models:
        if model not in PRELOADED_MODELS:
            get_pipeline_optimized(task, model, use_quantization=QUANTIZATION_AVAILABLE)
            PRELOADED_MODELS.add(model)

    print("✅ 사전 로딩 완료!\n")

# 8. 최적화된 텍스트 분류 함수
def classify_text_optimized(model_name, text_input, labels_input):
    if not text_input or not labels_input:
        gr.Warning("텍스트와 라벨을 모두 입력하세요.")
        return {}

    pipe = get_pipeline_optimized("zero-shot-classification", model_name, use_quantization=QUANTIZATION_AVAILABLE)
    if pipe is None:
        return {}

    candidate_labels = [label.strip() for label in labels_input.split(',')]

    try:
        result = pipe(text_input, candidate_labels=candidate_labels)
        formatted_result = {
            label: round(score, 3)
            for label, score in zip(result['labels'], result['scores'])
        }
        return formatted_result
    except Exception as e:
        gr.Error(f"분류 중 오류: {e}")
        return {}

def generate_text_optimized(model_name, prompt, max_len):
    if not prompt:
        gr.Warning("프롬프트를 입력하세요.")
        return ""

    pipe = get_pipeline_optimized("text-generation", model_name, use_quantization=QUANTIZATION_AVAILABLE)
    if pipe is None:
        return ""

    try:
        if pipe.model.config.pad_token_id is None:
            pipe.model.config.pad_token_id = pipe.tokenizer.eos_token_id

        result = pipe(
            prompt,
            max_new_tokens=int(max_len),
            num_return_sequences=1,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3
        )
        return result[0]['generated_text']
    except Exception as e:
        gr.Error(f"생성 중 오류: {e}")
        return ""

def summarize_text_optimized(model_name, text_input, min_len, max_len):
    if not text_input:
        gr.Warning("요약할 원본 텍스트를 입력하세요.")
        return ""

    pipe = get_pipeline_optimized("summarization", model_name, use_quantization=QUANTIZATION_AVAILABLE)
    if pipe is None:
        return ""

    try:
        if getattr(pipe, "task", "") == "text-generation":
            if pipe.model.config.pad_token_id is None:
                pipe.model.config.pad_token_id = pipe.tokenizer.eos_token_id

            prompt = (
                "다음 글의 핵심을 3~5문장으로 간결하게 한국어 요약하세요.\n\n"
                f"[원문]\n{text_input}\n\n[요약]"
            )
            out = pipe(
                prompt,
                max_new_tokens=int(max_len),
                num_return_sequences=1,
                do_sample=True,
                temperature=0.7
            )
            return out[0]["generated_text"].split("[요약]")[-1].strip()

        result = pipe(
            text_input,
            min_length=int(min_len),
            max_length=int(max_len)
        )
        return result[0]["summary_text"]
    except Exception as e:
        gr.Error(f"요약 중 오류: {e}")
        return ""

# 9. 메모리 정리 함수
def clear_memory():
    global PIPELINES_CACHE
    PIPELINES_CACHE.clear()
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gr.Info("💾 메모리 정리 완료!")
    return "메모리가 정리되었습니다."

# 10. Gradio UI 구성
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🚀 최적화된 Multi-Task AI 데모")
    gr.Markdown("각 탭을 눌러 원하는 AI 작업을 테스트해보세요.")

    with gr.Row():
        memory_btn = gr.Button("🧹 메모리 정리", size="sm", scale=1)
        memory_status = gr.Textbox(label="상태", scale=3, interactive=False)
        memory_btn.click(fn=clear_memory, outputs=memory_status)

    with gr.Tab("📰 텍스트 분류"):
        gr.Markdown("## Zero-Shot 텍스트 분류")

        model_cls = gr.Dropdown(label="분류 모델 선택", choices=cls_models, value=cls_models[0])
        with gr.Row():
            text_cls = gr.Textbox(label="분류할 텍스트", placeholder="예: 오늘 코스피 지수가 5% 급등했습니다.", scale=3)
            labels_cls = gr.Textbox(label="후보 라벨 (쉼표로 구분)", value="정치, 경제, 사회, IT, 스포츠", scale=2)

        output_cls = gr.Label(label="분류 결과 (확률)")

        with gr.Row():
            btn_cls = gr.Button("⚡ 분류 실행", variant="primary", scale=4)
            clear_btn_cls = gr.ClearButton(value="🔄 초기화", components=[text_cls, labels_cls, output_cls], scale=1)

        btn_cls.click(fn=classify_text_optimized, inputs=[model_cls, text_cls, labels_cls], outputs=[output_cls])

    with gr.Tab("✍️ 텍스트 생성"):
        gr.Markdown("## AI 텍스트 생성")

        model_gen = gr.Dropdown(label="생성 모델 선택", choices=gen_models, value=gen_models[0])
        text_gen = gr.Textbox(label="프롬프트 (시작 단어)", placeholder="예: 대한민국의 수도는")
        slider_gen = gr.Slider(label="최대 생성 토큰 수", minimum=10, maximum=200, value=50, step=10)

        output_gen = gr.Textbox(label="생성된 텍스트", lines=5, interactive=False)

        with gr.Row():
            btn_gen = gr.Button("⚡ 생성 시작", variant="primary", scale=4)
            clear_btn_gen = gr.ClearButton(value="🔄 초기화", components=[text_gen, slider_gen, output_gen], scale=1)

        btn_gen.click(fn=generate_text_optimized, inputs=[model_gen, text_gen, slider_gen], outputs=[output_gen])

    with gr.Tab("📑 텍스트 요약"):
        gr.Markdown("## AI 텍스트 요약")

        model_sum = gr.Dropdown(label="요약 모델 선택", choices=sum_models, value=sum_models[0])
        text_sum = gr.Textbox(label="요약할 원본 텍스트", lines=10, placeholder="여기에 긴 뉴스 기사나 문서를 붙여넣으세요...")
        with gr.Row():
            slider_min_sum = gr.Slider(label="최소 요약 길이", minimum=10, maximum=100, value=20, step=5)
            slider_max_sum = gr.Slider(label="최대 요약 길이", minimum=50, maximum=300, value=100, step=10)

        output_sum = gr.Textbox(label="요약된 텍스트", lines=5, interactive=False)

        with gr.Row():
            btn_sum = gr.Button("⚡ 요약 실행", variant="primary", scale=4)
            clear_btn_sum = gr.ClearButton(value="🔄 초기화", components=[text_sum, slider_min_sum, slider_max_sum, output_sum], scale=1)

        btn_sum.click(fn=summarize_text_optimized, inputs=[model_sum, text_sum, slider_min_sum, slider_max_sum], outputs=[output_sum])

# 11. 사전 로딩 실행
preload_models()

# 12. Gradio 앱 실행
print("\n🎉 Gradio 앱 실행 준비 완료!")
demo.launch(debug=True, share=True)